# 1,インストール・インポート

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#ここで必要なもの全てインストール
!pip install japanize-matplotlib  
!pip install ydata-profiling 
!pip install sweetviz
!pip install catboost

In [ ]:
#標準装備のnumpy2.0ではsweetvizが動かないためバージョンをさげる
!pip install --force-reinstall numpy==1.26.4 #保存時のみrun allができなくくてエラーが出るのでコメントアウトする


In [10]:
# モジュールのインポート
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import japanize_matplotlib
import matplotlib.pyplot as plt  # データ可視化のための基本的なグラフ描画ライブラリ
import seaborn as sns  # 高機能な統計グラフを描画するライブラリ
import sweetviz as sv #sweetvizを用いるため
import lightgbm as lgb
import xgboost as xgb
from scipy import stats#サイパイでウィルコクソンの順位和検定を行うため
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレストによる分類器
from sklearn.model_selection import StratifiedKFold  # 層化K分割交差検証を行うクラス
from ydata_profiling import ProfileReport#YData-ProfilingというEDAツールを使うため
from catboost import CatBoostClassifier#catboostを使うため
from sklearn.model_selection import train_test_split#データを学習用と検証用にわける
from sklearn.linear_model import LogisticRegression#ロジスティック回帰モデルをベースラインモデルとして作るため
from sklearn.preprocessing import StandardScaler#標準化するため
from sklearn.pipeline import Pipeline#前処理とモデルをまとめて順番通りに自動でやってくれる箱
from sklearn.model_selection import cross_val_score#ロジスティック回帰モデル用のデータを分けるため
from matplotlib.ticker import MultipleLocator#一定間隔でメモリをおく
from scipy.stats import mannwhitneyu#ウィルコクソンの順位和検定のため
from sklearn.cluster import KMeans#k-means法を用いるため　これでクラスタリングを行う
from sklearn.experimental import enable_iterative_imputer#ここでIterativeImputerが「実験的機能を使います」と宣言する。これがないと次でエラーがでる
from sklearn.impute import IterativeImputer#欠損値補完でIterativeImputerを使用するため

In [3]:
#セッション再起動してからバージョンを下げることができてるかの確認
print(np.__version__)

1.26.4


# 2,データの概要

In [5]:
train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [16]:
train.head(5)




,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
test.describe()

## カラム名の和訳(英語では概要をつかみにくいので)

番号,カラム名,日本語訳・意味<br>
0,Id,物件ID<br>
1,MSSubClass,建物の等級（1階建て、2階建て、新築など）<br>
2,MSZoning,用途地域区分（商業、住宅など）<br>
3,LotFrontage,道路に接している長さ（フィート）<br>
4,LotArea,敷地面積（平方フィート）<br>
5,Street,道路の舗装状態（砂利、舗装）<br>
6,Alley,路地へのアクセスの種類<br>
7,LotShape,敷地の形状（規則的、不規則）<br>
8,LandContour,土地の平坦さ<br>
9,Utilities,利用可能な公共サービス（水道、電気など）<br>
10,LotConfig,敷地の区画構成（角地など）<br>
11,LandSlope,土地の傾斜<br>
12,Neighborhood,エイムス市内の地区名（立地）<br>
13,Condition1,幹線道路や鉄道への近接性<br>
14,Condition2,幹線道路や鉄道への近接性（複数ある場合）<br>
15,BldgType,建物の種類（戸建て、タウンハウスなど）<br>
16,HouseStyle,住宅のスタイル（平屋、2階建てなど）<br>
17,OverallQual,全体の素材と仕上げの品質（1-10評価）<br>
18,OverallCond,全体的な状態の評価（1-10評価）<br>
19,YearBuilt,建設年<br>
20,YearRemodAdd,改築・リフォーム年<br>
21,RoofStyle,屋根の形状<br>
22,RoofMatl,屋根の素材<br>
23,Exterior1st,外壁材1<br>
24,Exterior2nd,外壁材2<br>
25,MasVnrType,外装用石積みの種類<br>
26,MasVnrArea,外装用石積みの面積<br>
27,ExterQual,外装材の品質評価<br>
28,ExterCond,外装材の状態評価<br>
29,Foundation,基礎の種類（コンクリートなど）<br>
30,BsmtQual,地下室の高さ（品質評価）<br>
31,BsmtCond,地下室の全体的な状態<br>
32,BsmtExposure,地下室の露出具合（外光、換気）<br>
33,BsmtFinType1,地下室の仕上げ評価1<br>
34,BsmtFinSF1,地下室の仕上げ面積1<br>
35,BsmtFinType2,地下室の仕上げ評価2<br>
36,BsmtFinSF2,地下室の仕上げ面積2<br>
37,BsmtUnfSF,地下室の未仕上げ面積<br>
38,TotalBsmtSF,地下室の総面積<br>
39,Heating,暖房の種類<br>
40,HeatingQC,暖房の品質と状態<br>
41,CentralAir,セントラルエアコンの有無（Y/N）<br>
42,Electrical,電気システム<br>
43,1stFlrSF,1階の面積<br>
44,2ndFlrSF,2階の面積<br>
45,LowQualFinSF,低品質な仕上げの面積（全フロア）<br>
46,GrLivArea,地上の居住面積合計（平方フィート）<br>
47,BsmtFullBath,地下室のフルバスルーム数<br>
48,BsmtHalfBath,地下室のハーフバスルーム数<br>
49,FullBath,地上のフルバスルーム数<br>
50,HalfBath,地上のハーフバスルーム数<br>
51,BedroomAbvGr,地上のベッドルーム数<br>
52,KitchenAbvGr,地上のキッチン数<br>
53,KitchenQual,キッチンの品質評価<br>
54,TotRmsAbvGrd,地上の総部屋数（バスルーム除く）<br>
55,Functional,住宅の機能性評価<br>
56,Fireplaces,暖炉の数<br>
57,FireplaceQu,暖炉の品質<br>
58,GarageType,ガレージの種類<br>
59,GarageYrBlt,ガレージの建設年<br>
60,GarageFinish,ガレージの内装仕上げ具合<br>
61,GarageCars,ガレージの収容台数<br>
62,GarageArea,ガレージの面積<br>
63,GarageQual,ガレージの品質<br>
64,GarageCond,ガレージの状態<br>
65,PavedDrive,ドライブウェイ（私道）の舗装状態<br>
66,WoodDeckSF,ウッドデッキの面積<br>
67,OpenPorchSF,オープンポーチの面積<br>
68,EnclosedPorch,囲いのあるポーチの面積<br>
69,3SsnPorch,三季用ポーチの面積<br>
70,ScreenPorch,スクリーンポーチ（網戸付き）の面積<br>
71,PoolArea,プールの面積<br>
72,PoolQC,プールの品質<br>
73,Fence,柵の品質<br>
74,MiscFeature,その他（テニスコート、納屋など）<br>
75,MiscVal,その他特徴の価値<br>
76,MoSold,売却月<br>
77,YrSold,売却年<br>
78,SaleType,販売の種類<br>
79,SaleCondition,販売条件<br>
80,SalePrice,販売価格（予測するターゲット）<br>

# 3,EDA

## (1)trainに関するEDA

In [17]:
#insull().sum()に追加で.to_stiringメソッドを追加することにより全行表示が可能(今回はしない)
#pandasではデータフレーム[True/Falseのリスト] と書くと、Trueの行だけを抽出するというルールがある
#[]により先に比較演算が行われる
train_null_df=train.isnull()
train_null=train.isnull().sum()
train_null=train_null[train_null>0]
print(train_null)

LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [18]:
#df名[df名　比較演算子　数]　→これによりdfの中でも条件を満たした行のみが取り出される
test_null=test.isnull().sum()
test_null=test_null[test_null>0]
print(test_null)


MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType       894
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64


In [19]:
#pd.concat([,])で二つのdfの接合が可能  axis=1は横方向に接続という意味
#.columnsメソッドでカラム名を指定
#.filina(0)で欠損値を0にする
#>0で0より大きいならTrue、0より小さにならFalse(bool型にしてる)
#.astype(int)メソッドによりTrueが1、falseが0になる(boola型をint型へ)
df_null=pd.concat([train_null,test_null],axis=1)
df_null.columns=['train_null','test_null']
df_null=(df_null.fillna(0)>-0).astype(int)

print(df_null)

              train_null  test_null
LotFrontage            1          1
Alley                  1          1
MasVnrType             1          1
MasVnrArea             1          1
BsmtQual               1          1
BsmtCond               1          1
BsmtExposure           1          1
BsmtFinType1           1          1
BsmtFinType2           1          1
Electrical             1          0
FireplaceQu            1          1
GarageType             1          1
GarageYrBlt            1          1
GarageFinish           1          1
GarageQual             1          1
GarageCond             1          1
PoolQC                 1          1
Fence                  1          1
MiscFeature            1          1
MSZoning               0          1
Utilities              0          1
Exterior1st            0          1
Exterior2nd            0          1
BsmtFinSF1             0          1
BsmtFinSF2             0          1
BsmtUnfSF              0          1
TotalBsmtSF            0    

In [ ]:
# #YData-Profilingの実行
# from ydata_profiling import ProfileReport
# profile = ProfileReport(train, title="ydata-profilingによるEDA結果", explorative=True)

# # 4. レポートをノートブック上に表示
# profile.to_notebook_iframe()

# # # 5. HTMLとして保存
# # profile.to_file("ydata_analysis.html")

In [ ]:
# #保存時はnumpyのバージョンを下げることが出ずにエラーがでるのでコメントアウト
# sweetviz_report = sv.analyze(train, target_feat='SalePrice')

# #ここでnotebook上の表示
# sweetviz_report.show_notebook()

# # # レポートをHTMLで出力して表示
# # sweetviz_report.show_html("Sweetviz_Report.html")

## (2)testに関するEDA

In [7]:
#YData-Profilingの実行
from ydata_profiling import ProfileReport
profile = ProfileReport(test, title="ydata-profilingによるEDA結果", explorative=True)



#  HTMLとして保存
profile.to_file("ydata_analysis_test.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 80/80 [00:02<00:00, 38.77it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
#保存時はnumpyのバージョンを下げることが出ずにエラーがでるのでコメントアウト
#今回は目的変数なし
sweetviz_report = sv.analyze(test)

# レポートをHTMLで出力して表示
sweetviz_report.show_html("Sweetviz_Report_test.html")

                                             |          | [  0%]   00:00 -> (? left)

Report Sweetviz_Report_test.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# 4,前処理

In [13]:
#.dropメソッドで()にカラム名とaxisで行か列を指定すると消える
train=train.drop('Id',axis=1)
test=test.drop('Id',axis=1)

KeyError: "['Id'] not found in axis"

## (1)trainの欠損値処理

trainの欠損値の一覧<br>
・LotFrontage has 259 (17.7%) missing valuesなので少ない<r>
・LotFrontage has 259 (17.7%) missing valuesなので少ない<br>
・Alley has 1369 (93.8%) missing valuesなのでこれは明らか欠損多い<br>
・MasVnrType has 872 (59.7%) missing valuesなので少し多い<br>
・BsmtQual has 37 (2.5%) missing valuesなので少ない<br>
・BsmtCond has 37 (2.5%) missing valuesなので少ない<br>
・BsmtExposure has 38 (2.6%) missing valuesなので少ない<br>	
・BsmtFinType1 has 37 (2.5%) missing valuesなので少ない<br>
・BsmtFinType2 has 38 (2.6%) missing valuesなので少ない<br>
・FireplaceQu has 690 (47.3%) missing valuesなので少し多い<br>
・GarageType has 81 (5.5%) missing valuesなので少ない<br>
・GarageYrBlt has 81 (5.5%) missing valuesなので少ない	<br>
・GarageFinish has 81 (5.5%) missing valuesなので少ない	<br>
・GarageQual has 81 (5.5%) missing valuesなので少ない<br>
・GarageCond has 81 (5.5%) missing valuesなので少ない<br>
・PoolQC has 1453 (99.5%) missing values多い	<br>
・Fence has 1179 (80.8%) missing values	多い<br>
・MiscFeature has 1406 (96.3%) missing 多いの<br>

In [ ]:
# #はじめ欠損が多いもののみが意図的に欠損させたものと思ってた時のコード
# # 調べたいカラムのリスト
# target_cols = ['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

# for col in target_cols:
#     print(f"{col} の中身")
#     # .value_countsメソッドで種類を数えることが可能  dropna=Falseで欠損値込みで数える
#     print(train[col].value_counts(dropna=False))
#     print('\n')#\nで改行が可能
    

### ①trainの欠損値処理の確定
欠損値の割合が大きいものはその欠損自体に意味があると仮定しあえて欠損値処理を行わない　<br>
「うっかり忘れ」か「あえて（無し）」かの区別がつかないのが、IterativeImputerの弱点であり、うっかり忘れたときに強い。<br>
ただし仮にそのnanが0という意味だったらIterativeImputerでは0という予測はできない<br><br>


ここで欠損値が多いと指定したものは意図的であるものが高いと思い仕様書を用いて調べてみたが本当に欠損しているものはLotFrontageだけだった<br>

---欠損値処理の方法---<br>
①LotFrontageにはIterativeImputerで欠損値処理<br>
②その他のint,float型データは0を代入して欠損値処理<br>
③その他のカテゴリデータはNoneをだいにゅうすて欠損値処理<br>

### ②trainの欠損値処理の実行

In [21]:
#まず欠損してる列の型をわける
# 1. チェックしたいカラムのリスト
target_cols = [
    'LotFrontage', 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 
    'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 
    'GarageCond', 'PoolQC', 'Fence', 'MiscFeature','MasVnrArea','Electrical'
]

# 2. 数値型と文字型に分ける
numeric_cols = train[target_cols].select_dtypes(include=['number']).columns.tolist()
object_cols = train[target_cols].select_dtypes(include=['object']).columns.tolist()

# 3. 結果を表示
print("数値型（0や予測値を入れる）")
print(numeric_cols)
print("\n文字型（'None'を入れる）")
print(object_cols)

数値型（0や予測値を入れる）
['LotFrontage', 'GarageYrBlt', 'MasVnrArea']

文字型（'None'を入れる）
['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'Electrical']


In [22]:
#1,LotFrontageをIterativeImputerで欠損値補完
#この際ヒントをとってくるためLotFrontageと相関が高いカラムをsweetvizから取ってくる
# 1. Sweetvizで確認した相関の高い4項目 + 埋めたい項目をリスト化 
target_and_hints = ['LotFrontage', 'LotArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea']

# 2. IterativeImputerの設定　ここで学習
it_imp = IterativeImputer(max_iter=10, random_state=42)

# 3. 補完の実行 ここで実行
# 指定した5つの列のデータセットを渡し、LotFrontageの欠損を予測して上書きします
train[target_and_hints] = it_imp.fit_transform(train[target_and_hints])

# 4. 確認
print(f"LotFrontageの欠損数: {train['LotFrontage'].isnull().sum()}")

LotFrontageの欠損数: 0


In [36]:
#2数値の型であるGarageYrBltとMasVnrAreaを0で補完する
#fillna()メソッドは指定した値で欠損値を埋めるという意味
tran=['GarageYrBlt','MasVnrArea']
train[tran]=train[tran].fillna(0)

#isnullで欠損があるかないかの0(False)と1(True)混合のdfを作ってsumでその欠損数を数える
print(f"欠損数: {train[tran].isnull().sum()}")

欠損数: GarageYrBlt    0
MasVnrArea     0
dtype: int64


In [24]:
#3文字型の欠損をNoneで代入
#先にlistでカラムをまとめたほうがpandasの処理が早くなる
trans=['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
       'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature','Electrical']

train[trans]=train[trans].fillna('None')
print(f"カテゴリデータの欠損数: {train[trans].isnull().sum()}")

カテゴリデータの欠損数: Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
Electrical      0
dtype: int64


In [25]:
# 表示する行数の上限を解除（None = 無制限）
pd.set_option('display.max_rows', None)

train.isnull().sum()

MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual 

## (2)trainのエンコーディング

### ①エンコーディングの方針
・targetencodingとはその要素と目的変数の関係を見てencodingする手法でただしこの際encodingしたい列を除外する。targetencodingは強力な手法だが上手にやらないとリークする可能性がある。<br>
・とりあえずベースモデル構築まではonehot、feature hashing、label、count、frequency、labelcount、Ordinal Encodingの中から選ぶ。<br>
・決定木モデルをメインに使う予定なので不均衡データでも種類数がすくないとonehotを採用する<br>

In [26]:
#.columns.tolist()をすることでdfだったのをカラム名だけにしたlistを取得することができる
category = train.select_dtypes(include=['object']).columns.tolist()


In [27]:
#.nunique()で種類数、value_counts()で実際の種類
for col in category:
    print(f"--- {col} ---")
    print(f"種類数: {train[col].nunique()}") # 何種類の値があるか  nuniqueでカラムの中身の種類数がわかる
    print(train[col].value_counts()) # value_countsメソッドで実際の種類が分かる
    print("\n")

--- MSZoning ---
種類数: 5
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64


--- Street ---
種類数: 2
Street
Pave    1454
Grvl       6
Name: count, dtype: int64


--- Alley ---
種類数: 3
Alley
None    1369
Grvl      50
Pave      41
Name: count, dtype: int64


--- LotShape ---
種類数: 4
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: count, dtype: int64


--- LandContour ---
種類数: 4
LandContour
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: count, dtype: int64


--- Utilities ---
種類数: 2
Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64


--- LotConfig ---
種類数: 5
LotConfig
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: count, dtype: int64


--- LandSlope ---
種類数: 3
LandSlope
Gtl    1382
Mod      65
Sev      13
Name: count, dtype: int64


--- Neighborhood ---
種類数: 25
Neighborhood
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert   

完全に優越のある、順位のある以下のカラムはordinalencoding
categoryというカテゴリデータのlistの中から

BsmtCond

HeatingQC

KitchenQual

FireplaceQu

GarageQual

GarageCond

PoolQC

をordinalencoding

In [28]:
# 1. Ordinal Encoding（順序あり）にするカラムのリスト定義
# 標準的な品質評価 (Ex, Gd, TA...) のグループ
ordinal_qual_cols = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 
    'HeatingQC', 'KitchenQual', 'FireplaceQu', 
    'GarageQual', 'GarageCond', 'PoolQC'
]
# 形状 (Reg, IR1...) のグループ
ordinal_shape_cols = ['LotShape']

# すべてのOrdinal系カラムをまとめたリスト（後で除外するため） これを省いたものをonehot
all_ordinal_cols = ordinal_qual_cols + ordinal_shape_cols


# 2. マッピング辞書（変換ルール）の作成 辞書型
# 品質用: Ex(5) > Gd(4) > TA(3) > Fa(2) > Po(1) > None(0)
qual_map = {
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 
    'None': 0, 'NA': 0
}

# 形状用: Reg(3) > IR1(2) > IR2(1) > IR3(0)
shape_map = {
    'Reg': 3, 'IR1': 2, 'IR2': 1, 'IR3': 0
}


# 3. Ordinal Encoding の実行
# 注意: マップする前に欠損値(NaN)を 'None' という文字列に埋めます
# これにより、「地下室なし」や「ガレージなし」が数値の 0 に変換されます

# 品質系の変換 map関数はあり規則に基づいて一気に置き換える手法
for col in ordinal_qual_cols:
    train[col] = train[col].fillna('None').map(qual_map)

# 形状系の変換
for col in ordinal_shape_cols:
    train[col] = train[col].fillna('None').map(shape_map)


# 4. 残りのカテゴリ変数を One-Hot Encoding
# 元の `category` リストから、今変換した Ordinal カラムを除外する 内包表記(結果代入用変数、ループ、条件)
remaining_cat_cols = [c for c in category if c not in all_ordinal_cols]

# 残りだけを One-Hot Encoding
train = pd.get_dummies(train, columns=remaining_cat_cols, drop_first=False)

# 確認
print("--- 処理完了 ---")
print(f"Ordinal変換したカラム数: {len(all_ordinal_cols)}")
print(f"One-Hot変換したカラム数: {len(remaining_cat_cols)}")
print(f"現在の全カラム数: {train.shape[1]}")

# ちゃんと数値になっているかチラ見
train[ordinal_qual_cols + ordinal_shape_cols].head()

--- 処理完了 ---
Ordinal変換したカラム数: 11
One-Hot変換したカラム数: 32
現在の全カラム数: 261


,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC,LotShape
0,4,3,4,3,5,4,0,3,3,0,3
1,3,3,4,3,5,3,3,3,3,0,3
2,4,3,4,3,5,4,3,3,3,0,2
3,3,3,3,4,4,4,4,3,3,0,2
4,4,3,4,3,5,4,3,3,3,0,2


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 261 entries, MSSubClass to SaleCondition_Partial
dtypes: bool(213), float64(7), int64(41)
memory usage: 851.3 KB


## (3)testの欠損値処理

欠損値補完: Trainで使った it_imp (IterativeImputer) を使って、Testデータを補完してください（fitはせずtransformのみ）。

One-Hot Encodingの罠: pd.get_dummies をTrainとTest別々に実行すると、**「TrainにはあるがTestにはないカテゴリ」**があった場合、カラム数（列の数）が合わなくなり、モデルに入れた瞬間にエラーになります。

解決策: TrainとTestを一度結合してから get_dummies するか、One-Hot Encoderなどのライブラリを使うのが安全ですが、まずは簡単な方法として、処理後に align を使う方法などを調べると良いでしょう。

In [30]:
# 1. 欠損値があるカラム名だけをリスト形式で取得
#.columnsでカラム名だけ抽出、.any()で1個でもTrueだったら持ってくる、.tolistでリスト型に変換
test_missing_cols = test.columns[test.isnull().any()].tolist()

# 2. ついでに「いくつ欠損しているか」も確認（これが見やすいです）
test_missing_info = test[test_missing_cols].isnull().sum()

print("--- 欠損があるカラムのリスト ---")
print(test_missing_cols)
print("\n--- 欠損数 ---")
print(test_missing_info)

--- 欠損があるカラムのリスト ---
['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']

--- 欠損数 ---
MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType       894
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
Gar

### <概要>補完方法<br>

他の値がほぼ同じ値→最頻値<br>
特に関係がない&外れ値がある→中央値<br>
特に関係がない&きれいな分布→平均値<br>
他の物と関係が深くある→IterativeImputer<br>

### ①意図的でないものの欠損補完

仕様書を見て以下のカラムが意図的な欠損でないと判断した。
また、EDAを行いデータの分布などを見て補完方法を決めていく(sweetvizより判断)

MSZoning: 用途地域（最頻値RLで埋める）

LotFrontage: 道路に接する間口（Trainと同様にIterativeImputer）

Utilities: インフラ設備（最頻値AllPubで埋める）

Exterior1st: 外壁材1（IterativeImputer,Exterior2ndとYearRemodAddとGarageAreaを使う）

Exterior2nd: 外壁材2（IterativeImputer、Exterior1stとYearBuiltとYearRemodAddを使う）

KitchenQual: キッチンの品質（最頻値1で埋める）

Functional: 機能性（最頻値Typで埋める）

SaleType: 販売タイプ（最頻値WLで埋める）

In [33]:
#欠損方法は自分で考えたが以下のコードはgeminiのものをコピペした
#ポイントはカテゴリデータのままではIterativeImputerは動かないのでenumrate関数で数を与える(ラベルエンコーディングと似てる)
# ==========================================
# 1. 最頻値（固定値）でサクッと埋めるグループ
# ==========================================
test['MSZoning'] = test['MSZoning'].fillna('RL')
test['Utilities'] = test['Utilities'].fillna('AllPub')
test['Functional'] = test['Functional'].fillna('Typ')

# ※補足の通り、KitchenQualは'TA'、SaleTypeは'WD'に修正してあります
test['KitchenQual'] = test['KitchenQual'].fillna('TA')
test['SaleType'] = test['SaleType'].fillna('WD')

# ==========================================
# 2. LotFrontage の予測補完（Trainと同じ手法）
# ==========================================
it_imp_lot = IterativeImputer(max_iter=10, random_state=42)
# Trainの時と同じヒント変数（面積系）を使用
lot_features = ['LotFrontage', 'LotArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea']
test[lot_features] = it_imp_lot.fit_transform(test[lot_features])

# ==========================================
# 3. Exterior1st と Exterior2nd の予測補完
# ==========================================
# ※文字データはそのままIterativeImputerに入れられないため、一時的に数字に変換して予測し、元に戻します

# ① 文字を数字に変換する辞書を作成
#dropnaで欠損値を含む行や列をまるごと削除
ext1_unique = test['Exterior1st'].dropna().unique()
ext2_unique = test['Exterior2nd'].dropna().unique()

ext1_map = {val: i for i, val in enumerate(ext1_unique)}
ext2_map = {val: i for i, val in enumerate(ext2_unique)}

# ② 数字を文字に戻すための逆引き辞書を作成
#enumerate関数は文字列に数を与える
ext1_inv_map = {i: val for i, val in enumerate(ext1_unique)}
ext2_inv_map = {i: val for i, val in enumerate(ext2_unique)}

# ③ 一時的に数字に変換したカラムを作成（NaNはNaNのまま）
#mapは変換したいときに辞書を参照して変換する関数
test['Ext1_num'] = test['Exterior1st'].map(ext1_map)
test['Ext2_num'] = test['Exterior2nd'].map(ext2_map)

# ④ IterativeImputer で予測
it_imp_ext = IterativeImputer(max_iter=10, random_state=42)

# Exterior1st の補完（ご指定のヒントを使用）
ext1_cols = ['Ext1_num', 'Ext2_num', 'YearRemodAdd', 'GarageArea']
test[ext1_cols] = it_imp_ext.fit_transform(test[ext1_cols])

# Exterior2nd の補完（ご指定のヒントを使用）
ext2_cols = ['Ext2_num', 'Ext1_num', 'YearBuilt', 'YearRemodAdd']
test[ext2_cols] = it_imp_ext.fit_transform(test[ext2_cols])

# ⑤ 予測された数値を四捨五入して、元の文字データに戻して穴埋め
test['Exterior1st'] = test['Exterior1st'].fillna(test['Ext1_num'].round().map(ext1_inv_map))
test['Exterior2nd'] = test['Exterior2nd'].fillna(test['Ext2_num'].round().map(ext2_inv_map))

# ⑥ 計算用に使った一時的な数値カラムをお掃除（削除）
test.drop(['Ext1_num', 'Ext2_num'], axis=1, inplace=True)

print("テストデータの意図的ではない欠損の補完がすべて完了しました！")

テストデータの意図的ではない欠損の補完がすべて完了しました！


### ②意図的であるものの欠損補完

In [31]:
#意図的でない欠損の補完の実行
#まず欠損してる列の型をわける
# 1. チェックしたいカラムのリスト
target_cols = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
    'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 
    'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType',
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 
    'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea'
]



# 2. 数値型と文字型に分ける
numeric_col = test[target_cols].select_dtypes(include=['number']).columns.tolist()
object_col = test[target_cols].select_dtypes(include=['object']).columns.tolist()

# 3. 結果を表示
print("数値型（0や予測値を入れる）")
print(numeric_col)
print("\n文字型（'None'を入れる）")
print(object_col)

数値型（0や予測値を入れる）
['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

文字型（'None'を入れる）
['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType']


In [34]:
#数値のデータの補完

test[numeric_col]=test[numeric_col].fillna(0)

#isnullで欠損があるかないかの0(False)と1(True)混合のdfを作ってsumでその欠損数を数える
print(f"欠損数: {test[numeric_col].isnull().sum()}")

欠損数: MasVnrArea      0
BsmtFinSF1      0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
GarageCars      0
GarageArea      0
dtype: int64


In [37]:
#カテゴリデータ」の補完
test[object_col]=test[object_col].fillna('None')
print(f"カテゴリデータの欠損数: {test[object_col].isnull().sum()}")

カテゴリデータの欠損数: Alley           0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
MasVnrType      0
dtype: int64


In [39]:
train.isnull().sum()

MSSubClass               0
LotFrontage              0
LotArea                  0
LotShape                 0
OverallQual              0
OverallCond              0
YearBuilt                0
YearRemodAdd             0
MasVnrArea               0
ExterQual                0
ExterCond                0
BsmtQual                 0
BsmtCond                 0
BsmtFinSF1               0
BsmtFinSF2               0
BsmtUnfSF                0
TotalBsmtSF              0
HeatingQC                0
1stFlrSF                 0
2ndFlrSF                 0
LowQualFinSF             0
GrLivArea                0
BsmtFullBath             0
BsmtHalfBath             0
FullBath                 0
HalfBath                 0
BedroomAbvGr             0
KitchenAbvGr             0
KitchenQual              0
TotRmsAbvGrd             0
Fireplaces               0
FireplaceQu              0
GarageYrBlt              0
GarageCars               0
GarageArea               0
GarageQual               0
GarageCond               0
W

## (4)testのエンコーディング